In [121]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [122]:
data = pd.read_csv('noticias-economicas-sample.csv')


In [123]:
#ignoramos noticias muy cortas y nos quedamos con 1000

df = data.copy()
df['ntext'] = df['text'].str.split().str.len()
df = df[df['ntext'] > 200]
df.reset_index(inplace=True, drop=True)
df = df.loc[:1000]

In [124]:
df

,url,base_url,authors,title,summary,text,publish_date,top_image_url,keywords,insert_date,ntext
0,https://www.expansion.com/economia/2020/11/24/...,https://www.expansion.com/mercados/cronica-bol...,Christian Marquardt,La confianza empresarial de Alemania vuelve a ...,CHRISTIAN MARQUARDT / POOL EFELa confianza emp...,Angela Merkel ayer en Berlin tras la cumbre te...,2020-11-24 00:00:00,https://phantom-expansion.unidadeditorial.es/a...,"noviembre, confianza, empresarial, económica, ...",2020-11-24 12:00:00,567
1,https://www.expansion.com/blogs/quemada/2020/1...,https://www.expansion.com/mercados/cronica-bol...,NaN,El Covid dejará 10 cambios que te afectarán,NaN,\n\n\n\n\nEnrique Quemada Clariana\nFusiones y...,2020-11-06 01:00:00,https://e00-expansion.uecdn.es/assets/desktop/...,"dejará, covid, cambios, afectarán",2020-11-07 01:00:00,686
2,https://cincodias.elpais.com/cincodias/2020/12...,https://cincodias.elpais.com/tag/ibex_35/,"Ediciones Cinco Días, Getty Images",Quiero ser mi propio jefe,¿Cuántas veces no ha querido ser su propio jef...,¿Cuántas veces no ha querido ser su propio jef...,2020-12-10 00:00:00,https://d500.epimg.net/cincodias/imagenes/2020...,"ser, quiero, propio, negocio, jefe",2020-12-14 12:00:00,585
3,https://elpais.com/sociedad/2020-12-04/illa-es...,https://cincodias.elpais.com/tag/ibex_35/,"Pedro Gorospe, Javier Etxezarreta",Illa estima que entre mayo y junio pueden esta...,Javier Etxezarreta / EFEEl ministro de Sanidad...,"El ministro de Sanidad, Salvador Illa, momento...",2020-12-04 00:00:00,https://imagenes.elpais.com/resizer/N92W5q5ZxY...,"mayo, illa, pueden, millones, estima, vacunada...",2020-12-04 12:00:00,586
4,https://www.bolsamania.com/noticias/economia/c...,https://www.bolsamania.com/indice/IBEX-35/noti...,Web Financial Group,El Congreso aprueba los Presupuestos con 188 v...,El Pleno del Congreso ha aprobado el proyecto ...,El Pleno del Congreso ha aprobado el proyecto ...,NaN,https://img2.s3wfg.com/web/img/images_uploaded...,"millones, congreso, presupuestos, favor, aprue...",2020-12-03 12:00:00,790
...,...,...,...,...,...,...,...,...,...,...,...
996,https://www.expansion.com/valencia/2020/10/13/...,https://www.expansion.com/mercados/cronica-bol...,Artur Zanon,Los proveedores de snacks de Mercadona inviert...,"EMPRESA LIVEN, EN BERGA, BARCELONA ARTUR ZANON...","EMPRESA LIVEN, EN BERGA, BARCELONA ARTUR ZANON...",2020-10-13 02:00:00,https://phantom-expansion.unidadeditorial.es/4...,"euros, snacks, millones, mercadona, invierten,...",2020-10-13 14:00:00,396
997,http://cincodias.elpais.com/cincodias/2021/01/...,https://cincodias.elpais.com/tag/ibex_35/,Alfonso Simón Ruiz,Grifols prueba en España su tratamiento frente...,La multinacional catalana Grifols está en ese ...,Mientras avanzan las vacunas para inmunizar a ...,2021-01-18 00:00:00,https://d500.epimg.net/cincodias/imagenes/2021...,"covid19, grifols, frente, españa, tratamiento,...",2021-01-18 12:00:00,610
998,https://www.bolsamania.com/noticias/analisis-t...,https://www.bolsamania.com/indice/IBEX-35/noti...,"Web Financial Group, Jm. Rodríguez, César Vidal",Meliá Hotels: ¿a por el hueco bajista semanal ...,El sector del turismo está subiendo con violen...,El sector del turismo está subiendo con violen...,NaN,https://img5.s3wfg.com/web/img/images_uploaded...,"vamos, marzo, hotels, bajista, meliá, semanal,...",2020-11-12 13:00:00,201
999,http://cincodias.elpais.com/cincodias/2021/01/...,https://cincodias.elpais.com/tag/ibex_35/,José Luis Sanz,Cómo cambiar el nombre de los dispositivos blu...,La sección bluetooth de nuestros iPhone ha ido...,La sección bluetooth de nuestros iPhone ha ido...,2021-01-29 00:00:00,https://d500.epimg.net/cincodias/imagenes/2021...,"dispositivos, cómo, iphone, bluetooth, nombre,...",2021-01-29 12:00:00,457


Primero, una vectorización en base al conteo de palabras de cada texto de la base

In [125]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])

In [126]:
X.todense().shape

(1001, 44796)

## Lemmatizing y stopwords

Aparentemente, el texto simple contiene demasiadas palabras únicas y _stopwords_ lo que devuelve una matrix con demasiados ceros (sparse), lo que podría resultar en algo indeseado. Para mejorar este problema, preprocesaremos los datos eliminando stopwords o palabras vacías (preposiciones y artículos principalmente) y haciendo _lemmatizing_ (cortar las palabras a su versión no flexionada, por ejemplo: flexionado $\rightarrow$ flexionar.

In [127]:
nlp = spacy.load('es_core_news_md')

In [128]:
stop_words = spacy.lang.es.stop_words.STOP_WORDS
len(stop_words)

521

In [129]:
df.loc[:, 'text2'] = df.text.apply(lambda text: " ".join(token.lemma_ for token in nlp(text) if not token.is_stop))

In [130]:
oracion = "El otro encuentro que falta por desarrollarse es el del líder Huachipato frente al colista Magallanes"

oracionlimpia = [x.lemma_ for x in nlp(oracion) if not x.is_stop]
oracionlimpia

['encuentro',
 'faltar',
 'desarrollar él',
 'líder',
 'Huachipato',
 'frente',
 'colista',
 'Magallanes']

In [131]:
df[['text', 'text2']]

,text,text2
0,Angela Merkel ayer en Berlin tras la cumbre te...,Angela Merkel ayer Berlin cumbre telemático G-...
1,\n\n\n\n\nEnrique Quemada Clariana\nFusiones y...,\n\n\n\n\n Enrique Quemada Clariana \n Fusione...
2,¿Cuántas veces no ha querido ser su propio jef...,¿ querer jefe ? ¿ arrepentir lanzar él montar ...
3,"El ministro de Sanidad, Salvador Illa, momento...","ministro Sanidad , Salvador Illa , momento rue..."
4,El Pleno del Congreso ha aprobado el proyecto ...,Pleno Congreso aprobar proyecto Presupuestos G...
...,...,...
996,"EMPRESA LIVEN, EN BERGA, BARCELONA ARTUR ZANON...","empresa LIVEN , BERGA , BARCELONA ARTUR ZANON ..."
997,Mientras avanzan las vacunas para inmunizar a ...,avanzar vacuna inmunizar población SARS-CoV-2 ...
998,El sector del turismo está subiendo con violen...,"sector turismo subir violencia . , conocer él ..."
999,La sección bluetooth de nuestros iPhone ha ido...,sección bluetooth iphone ir coger cuerpo año p...


In [132]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text2'])


In [133]:
M = X.todense()

In [134]:
M.shape

(1001, 32775)

In [135]:
def get_term(dict, search_index):
    return list(dict.keys())[list(dict.values()).index(search_index)]

In [136]:
get_term(vectorizer.vocabulary_, 8)

'0101'

Ahora, probemos el uso de pca sobre los datos vectorizados

In [137]:
U, S, VT = np.linalg.svd(M, full_matrices=False)

In [138]:
V = VT.T

s = np.diag(S)

In [139]:
Mhat = U[:, :1000] @ s[:1000, :1000] @ V[:,:1000].T
Mhatfull = U @ s @ V.T

In [140]:
Mhat.shape

(1001, 32775)

In [141]:
def show_summary(Mhat, doc, wordsn, vectorizer):
    row = pd.DataFrame(Mhat[doc,:])
    row = row.T.sort_values(0, ascending=False)
    row_i = row.reset_index()
    principales = list(row_i.loc[:wordsn,]['index'])
    for x in principales:
        print(get_term(vectorizer.vocabulary_, x))


In [142]:
doc = 10
wordsn = 20

In [143]:
print('pca')
print('\n')
show_summary(Mhat, doc, wordsn, vectorizer)
print('\n')
print('Full svd')
print('\n')
show_summary(Mhatfull, doc, wordsn, vectorizer)
print(df.loc[doc, 'text'])

pca


salud
zona
madrid
comunidad
básico
00
medida
sanidad
puente
santos
35
restricción
confinamiento
decreto
región
adoptar
regional
pública
transmisión
limitación
decidir


Full svd


salud
zona
comunidad
madrid
básico
00
medida
sanidad
puente
santos
35
restricción
confinamiento
decreto
adoptar
región
regional
pública
transmisión
limitación
decidir
La Comunidad de Madrid permanece cerrada perimetralmente desde las 00.00 horas de este viernes y continuará así hasta las 00.00 del próximo martes para impedir los desplazamientos durante el puente de Todos los Santos, confinamiento que ha entrado en vigor con la publicación del decreto regional en el 'Boletín Oficial de la Comunidad de Madrid'(BOCM).

La presidenta de la Comunidad de Madrid, Isabel Díaz Ayuso, ha firmado el decreto regional para limitar la entrada y salida de la región durante el Puente de Todos los Santos y el de la Almudena, aunque la decisión sobre este último se tomará en el Consejo Interterritorial de Salud del miérc

### PCA muestra algo de los documentos en pocas palabras, en algunos caso lo hace bien y en otras es más impreciso. Probemos con otro tipo de vectorizador basado en TF-IDF

In [144]:
vectorizer2 = TfidfVectorizer()
X2 = vectorizer2.fit_transform(df['text2'])
M2 = X2.todense()

In [145]:
U2, S2, VT2 = np.linalg.svd(M2, full_matrices=False)
V2 = VT2.T
s2 = np.diag(S2)
Mhat2 = U2[:, :1000] @ s2[:1000, :1000] @ V2[:,:1000].T

In [146]:
show_summary(Mhat2, doc, wordsn, vectorizer2)
print(df.loc[doc, 'text'])

salud
santos
00
zona
comunidad
básico
puente
madrid
sanidad
consejería
transmisión
pública
limitación
ciento
regional
decreto
adoptar
bocm
perimetralmente
tamames
andrada
La Comunidad de Madrid permanece cerrada perimetralmente desde las 00.00 horas de este viernes y continuará así hasta las 00.00 del próximo martes para impedir los desplazamientos durante el puente de Todos los Santos, confinamiento que ha entrado en vigor con la publicación del decreto regional en el 'Boletín Oficial de la Comunidad de Madrid'(BOCM).

La presidenta de la Comunidad de Madrid, Isabel Díaz Ayuso, ha firmado el decreto regional para limitar la entrada y salida de la región durante el Puente de Todos los Santos y el de la Almudena, aunque la decisión sobre este último se tomará en el Consejo Interterritorial de Salud del miércoles, que decidirá sobre la efectividad de las medidas que están adoptando las autonomías.

Además, la Consejería de Sanidad de la Comunidad de Madrid ha ampliado de 32 a 35 las zona

### Finalmente, el uso de bigramas para revisar si encuentra relaciones entre 2 palabras que juntas conforman un término único

In [147]:
vectorizer3 = TfidfVectorizer(ngram_range=(1, 2))
X3 = vectorizer3.fit_transform(df['text2'])
M3 = X3.todense()

U3, S3, VT3 = np.linalg.svd(M3, full_matrices=False)
V3 = VT3.T
s3 = np.diag(S3)
Mhat3 = U3[:, :1000] @ s3[:1000, :1000] @ V3[:,:1000].T

In [148]:
show_summary(Mhat3, doc, wordsn, vectorizer3)
print(df.loc[doc, 'text'])

básico salud
zona básico
puente santos
comunidad madrid
salud
santos
00
zona
comunidad
básico
puente
35 zona
región puente
decreto regional
madrid
sanidad
00 00
consejería sanidad
salud pública
consejería
transmisión
La Comunidad de Madrid permanece cerrada perimetralmente desde las 00.00 horas de este viernes y continuará así hasta las 00.00 del próximo martes para impedir los desplazamientos durante el puente de Todos los Santos, confinamiento que ha entrado en vigor con la publicación del decreto regional en el 'Boletín Oficial de la Comunidad de Madrid'(BOCM).

La presidenta de la Comunidad de Madrid, Isabel Díaz Ayuso, ha firmado el decreto regional para limitar la entrada y salida de la región durante el Puente de Todos los Santos y el de la Almudena, aunque la decisión sobre este último se tomará en el Consejo Interterritorial de Salud del miércoles, que decidirá sobre la efectividad de las medidas que están adoptando las autonomías.

Además, la Consejería de Sanidad de la Comun